In [1]:
pip install FastKmedoids --upgrade

  Using cached FastKmedoids-0.0.10-py3-none-any.whl.metadata (895 bytes)
Using cached FastKmedoids-0.0.10-py3-none-any.whl (12 kB)
Note: you may need to restart the kernel to use updated packages.


In [4]:
from FastKmedoids.models import FastKmedoidsGG

In [5]:
FastKmedoidsGG

FastKmedoids.models.FastKmedoidsGG